## Definition of Standard Data Structure

- Each ***variable*** has its own, **single** column and contain ***values***
- Each observation forms a row
- Each type of ***observational unit*** forms a table

### Example of messy datasets

In [2]:
import pandas as pd

messy_dict = {'':['John', 'Jane'], 'Treatment A': ['-', 16], 'Treatment B': [2, 11]}

messy_df = pd.DataFrame.from_dict(messy_dict)

messy_df

,,Treatment A,Treatment B
0,John,-,2
1,Jane,16,11


### Example of a tidy dataset

In [3]:
tidy_dict = {'Name':['John', 'John', 'Jane', 'Jane'], 'Treatment': ['A', 'B', 'A', 'B'], 'Result':['-', 2, 16, 11]}

tidy_df = pd.DataFrame.from_dict(tidy_dict)

tidy_df


,Name,Treatment,Result
0,John,A,-
1,John,B,2
2,Jane,A,16
3,Jane,B,11


## Basic Types of Messy Datasets and How to tidy them

### Type1 : Column headers are values, not variable names
**Using pd.melt to solve**

In [5]:
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re

#### Example 1:

In [7]:
df = pd.read_csv("pew-raw.csv")

In [9]:
df.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58
3,Catholic,418,617,732,670,638,1116
4,Dont know/refused,15,14,15,11,10,35


This dataset has rownames being different income range, or values of a variable named 'Income'

We use pandas melt function to reshape the dataset

In [15]:
tidy_df = pd.melt(df, ['religion'], var_name = 'income', value_name = 'freq')

tidy_df = tidy_df.sort_values(by = ['religion'])

tidy_df.head(10)



,religion,income,freq
0,Agnostic,<$10k,27
30,Agnostic,$30-40k,81
40,Agnostic,$40-50k,76
50,Agnostic,$50-75k,137
10,Agnostic,$10-20k,34
20,Agnostic,$20-30k,60
41,Atheist,$40-50k,35
21,Atheist,$20-30k,37
11,Atheist,$10-20k,27
31,Atheist,$30-40k,52


#### Example 2: Billboard Top 100 Dataset

**Description:**
It has information about a song from the moment it enters Billboard top 1000 and its weekly rank for the subsequent 75 weeks

In [59]:
df = pd.read_csv("billboard.csv", encoding = 'mac_latin2')
df.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Problems:**

1. The index of each week for the subsequent 75 weeks are all in the column header
2. Many NaN values for larger index of week because the song is no longer top 100 after this long time.

**Our Solution:**

1. Make all of the weeks' number into one single column which one row per each record
2. if no data(NaN) for a week, we do not create the row

In [60]:
# Melting
id_vars = ["year",
           "artist.inverted",
           "track",
           "time",
           "genre",
           "date.entered",
           "date.peaked"]

df = pd.melt(frame=df,id_vars=id_vars, var_name="week", value_name="rank")

# Formatting 
df["week"] = df['week'].str.extract('(\d+)', expand=False).astype(int)

# Cleaning out unnecessary rows
df = df.dropna()

# Create "date" columns
df['date'] = pd.to_datetime(df['date.entered']) + pd.to_timedelta(df['week'], unit='w') - pd.DateOffset(weeks=1)

df = df[["year", 
         "artist.inverted",
         "track",
         "time",
         "genre",
         "week",
         "rank",
         "date"]]
df = df.sort_values(ascending=True, by=["year","artist.inverted","track","week","rank"])

# Assigning the tidy dataset to a variable for future usage
billboard = df

df.head(10)

,year,artist.inverted,track,time,genre,week,rank,date
246,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,1,87.0,2000-02-26
563,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,2,82.0,2000-03-04
880,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,3,72.0,2000-03-11
1197,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,4,77.0,2000-03-18
1514,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,5,87.0,2000-03-25
1831,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,6,94.0,2000-04-01
2148,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,7,99.0,2000-04-08
287,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,1,91.0,2000-09-02
604,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,2,87.0,2000-09-09
921,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,3,92.0,2000-09-16


### Type2: Multiple Types(observational units)in One Table

**Identify the observational units and their attributes, create individual tables for each unit, make connect (1 way is to introduce foreign key in one table which is the primary key in another**

Looking at the previous table of billboard top 100 we cleaned, there are still a lot of repetition of rows regarding track, time, genre.

This broke the tidy data rule #3 which is 'Each type of observational unit forms a table'

Why?

In our table, the problem of repetition is caused by including both song's information and ranks' information in one table. Yeah, they are related; but it looks quite messy when they are put together.

**Solution:**
Let's create a table for song and rank respectively.

In [70]:
# for songs table

songs_cols = ['year', 'artist.inverted', 'track', 'time', 'genre']

songs = df[songs_cols].drop_duplicates()

songs = songs.reset_index(drop = True)

songs['song_id'] = songs.index

songs.head()

,year,artist.inverted,track,time,genre,song_id
0,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,0
1,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,1
2,2000,3 Doors Down,Kryptonite,3:53,Rock,2
3,2000,3 Doors Down,Loser,4:24,Rock,3
4,2000,504 Boyz,Wobble Wobble,3:35,Rap,4


In [73]:
# for rank table

ranks = pd.merge(billboard, songs, on = ['year', 'artist.inverted', 'track', 'time', 'genre'])

ranks = ranks[['song_id', 'date', 'rank']]

ranks.head(10)

,song_id,date,rank
0,0,2000-02-26,87.0
1,0,2000-03-04,82.0
2,0,2000-03-11,72.0
3,0,2000-03-18,77.0
4,0,2000-03-25,87.0
5,0,2000-04-01,94.0
6,0,2000-04-08,99.0
7,1,2000-09-02,91.0
8,1,2000-09-09,87.0
9,1,2000-09-16,92.0


### Type3: Multiple Variables stored in one column

**Creates columns for each variable for extracting appropriate information from the column that has multiple vatiables. The column creation may require pd.melt**

#### Example1: Tubercolosis Records from WHO

This datasets includes count of confirmed tuberculosis cases by country, year, age and sex.

In [92]:
df = pd.read_csv('tb-raw.csv')
df.head()

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,NaN,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,NaN,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,NaN,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,NaN,3.0


Note that these columns, m014, m1525,... are actually (sex, age lower bound, age upper bound)

Hence,

**Problems:**
   1. Multiple values contain in one columns (sex and age)
   2. Mix use of 0 and NaN. Distinction is important for this dataset


In [93]:
# Melting

df = pd.melt(frame = df, id_vars = ['country', 'year'], var_name = 'sex_and_age', value_name = 'cases')


In [94]:
# Extract Sex, Age
tmp_df = df['sex_and_age'].str.extract('(\D)(\d+)(\d{2})')
tmp_df.columns = ['sex', 'age_lower', 'age_upper']
tmp_df['age'] = tmp_df.age_lower + '-' + tmp_df.age_upper

# Merge
df = pd.concat([df, tmp_df], axis = 1)

# Drop unnecessary columns and row

df = df.drop(['sex_and_age', 'age_lower', 'age_upper'], axis = 1)
df = df.dropna()
df = df.sort_values(ascending = True, by = ['country', 'year', 'sex', 'age'])
df.head(10)

,country,year,cases,sex,age
0,AD,2000,0.0,m,0-14
10,AD,2000,0.0,m,15-24
20,AD,2000,1.0,m,25-34
30,AD,2000,0.0,m,35-44
40,AD,2000,0.0,m,45-54
50,AD,2000,0.0,m,55-64
81,AE,2000,3.0,f,0-14
1,AE,2000,2.0,m,0-14
11,AE,2000,4.0,m,15-24
21,AE,2000,4.0,m,25-34


### Type4: Variables are stored in both rows and columns

1. Melting to fix the issue with columns `pd.melt`
2. Pivot to fix the issue with rows `pd.pivot_tables()`

#### Example 1: Global Historical Climatology Network Dataset

This dataset represents the daily weather records for a weather station (MX17004) in Mexico for five months in 2010

In [126]:
df = pd.read_csv("weather-raw.csv")
df

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN,NaN
5,MX17004,2010,3,tmin,NaN,NaN,NaN,NaN,14.2,NaN,NaN,NaN
6,MX17004,2010,4,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MX17004,2010,4,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MX17004,2010,5,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,MX17004,2010,5,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Problem**
- Variables are stored in both rows(tmin, tmax) and columns(days)

In order to make this dataset tidy, we want to move the three misplaced variables (tmin, tmax and days) as three individual columns: tmin. tmax and date.

In [127]:
df = pd.melt(frame = df, id_vars = ['id', 'year', 'month', 'element'], var_name = 'day_raw', value_name = 'temp')

In [128]:
# Extract days
df['days'] = df['day_raw'].str.extract('d(\d)')
df.id.value_counts() # there are two ids even though they have the same number from our human eyes, might be an extra space due to data collection errors
df['id'] = 'MX17004' 
df.dtypes #df.days is of object type
df['days'] = df.days.astype(int)
df.dtypes

# Creating a date from the different columns

def create_date_from_year_month_day(row):
    return datetime.datetime(year=row["year"], month=int(row["month"]), day=row["days"])

df['date'] = df.apply(lambda row: create_date_from_year_month_day(row), axis = 1)
df = df.drop(['year',"month","days", "day_raw"], axis=1)
df = df.dropna()

df.head()



,id,element,temp,date
12,MX17004,tmax,27.3,2010-02-02
13,MX17004,tmin,14.4,2010-02-02
22,MX17004,tmax,24.1,2010-02-03
23,MX17004,tmin,14.4,2010-02-03
44,MX17004,tmax,32.1,2010-03-05


In [129]:
# unmelting the table to make the tmin and tmax be row header


df = df.pivot_table(index = ('id', 'date'), columns = 'element', values = 'temp')
df.reset_index(drop = False, inplace = True)
df

element,id,date,tmax,tmin
0,MX17004,2010-02-02,27.3,14.4
1,MX17004,2010-02-03,24.1,14.4
2,MX17004,2010-03-05,32.1,14.2


### Type 5: One Type in Multiple Tables

We need glob to get data in different files and concate them


#### Baby Names in Illinois

**Problem**
- Two files, 2014, and 2015 both contain the data we need
- The 'Year' variable is present in the file name

In order to load those different files into a single DataFrame, we can run a custom script that will append the files together. Furthermore, we’ll need to extract the “Year” variable from the file name.
In order to load those different files into a single DataFrame, we can run a custom script that will append the files together. Furthermore, we’ll need to extract the “Year” variable from the file name.

In [131]:
def extract_year(string):
    match = re.match(".+(\d{4})", string) 
    if match != None: return match.group(1)
    
allFiles = glob.glob("201*-baby-names-illinois.csv")
frame = pd.DataFrame()
df_list= []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    df.columns = map(str.lower, df.columns)
    df["year"] = extract_year(file_)
    df_list.append(df)
    
df = pd.concat(df_list)
df.head(5)

,rank,name,frequency,sex,year
0,1,Noah,837,Male,None
1,2,Alexander,747,Male,None
2,3,William,687,Male,None
3,4,Michael,680,Male,None
4,5,Liam,670,Male,None
